In [1]:
import pandas as pd
run=pd.read_csv('runs/run.monobert.ans_small.dev.trec',sep='\t',header=None,
                names=['qid','q0','pid','rank','score','tag'], usecols=['qid','pid','rank','score'])
print(run.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   qid     105000 non-null  int64  
 1   pid     105000 non-null  int64  
 2   rank    105000 non-null  int64  
 3   score   105000 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 3.2 MB
None


In [2]:
import numpy as np
from tqdm import tqdm
qidpid2idx = {}
array = []
with open('runs/run.monobert.ans_small.dev.bin','rb') as f:
    for line in tqdm(run.itertuples()):
        qidpid2idx[(line.qid,line.pid)]=len(array)
        array.append(np.frombuffer(f.read(1024*4),np.float32))
array=np.vstack(array)
print(array.shape)

105000it [00:00, 132638.16it/s]


(105000, 1024)


In [3]:
import torch
X = torch.FloatTensor(array).cuda()
Y = torch.exp(torch.FloatTensor(run.score.values)).unsqueeze(-1)
Y = torch.cat((Y,1-Y),dim=1).cuda()
X,Y

(tensor([[-0.6464, -0.9506,  1.0000,  ..., -0.2989,  0.8946, -0.8920],
         [-0.9502, -0.9819,  1.0000,  ..., -0.4000,  0.9394, -0.7937],
         [-0.9691, -0.9796,  1.0000,  ..., -0.6041,  0.9558, -0.8120],
         ...,
         [-0.8519, -0.8650, -0.9892,  ...,  0.4534, -0.8864,  0.7240],
         [-0.8595, -0.8771, -0.9916,  ...,  0.4298, -0.9025,  0.7257],
         [-0.8435, -0.8753, -0.9861,  ...,  0.4049, -0.8895,  0.7152]],
        device='cuda:0'),
 tensor([[9.9722e-01, 2.7757e-03],
         [9.9180e-01, 8.1974e-03],
         [9.8871e-01, 1.1288e-02],
         ...,
         [8.1602e-06, 9.9999e-01],
         [8.1043e-06, 9.9999e-01],
         [8.1000e-06, 9.9999e-01]], device='cuda:0'))

In [52]:
class classification(torch.nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        self.output = torch.nn.Sequential(torch.nn.Linear(1024,16),
#                                           torch.nn.BatchNorm1d(64),
                                          torch.nn.GELU(),
#                                           torch.nn.Dropout(p=0.1),
                                          torch.nn.Linear(16,2))
        self.prob = torch.nn.LogSoftmax(dim=1)
        
    def forward(self, X, Y):
        loss=(-Y*self.prob(self.output(X))).sum(dim=1).mean()
        return loss

train_X, dev_X = X[:5000,:], X[5000:,:]
train_Y, dev_Y = Y[:5000,:], Y[5000:,:]
model = classification().cuda()
optim = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, max_lr=0.01, steps_per_epoch=1000, epochs=1)
model.train()
for i in range(1000):
    optim.zero_grad()
    loss=model(train_X,train_Y)
    if (i+1)%50 == 0:
        model.eval()
        print(model(train_X,train_Y).item(),model(dev_X,dev_Y).item())
        model.train()
    loss.backward()
    optim.step()
    scheduler.step()

0.023394381627440453 0.013968189246952534
0.021132873371243477 0.012683595530688763
0.019474785774946213 0.011692256666719913
0.01929115317761898 0.011531731113791466
0.01922936923801899 0.011475265957415104
0.01920214854180813 0.011444511823356152
0.019190602004528046 0.011429235339164734
0.01918530836701393 0.011421655304729939
0.019182439893484116 0.011417590081691742
0.019180577248334885 0.011415163055062294
0.019179215654730797 0.01141355186700821
0.019178174436092377 0.011412392370402813
0.019177371636033058 0.011411515064537525
0.01917676255106926 0.011410841718316078
0.019176315516233444 0.011410340666770935
0.019176004454493523 0.011409984901547432
0.019175808876752853 0.011409755796194077
0.019175700843334198 0.011409630998969078
0.019175661727786064 0.011409582570195198
0.019175656139850616 0.011409573256969452


In [57]:
from sklearn.decomposition import PCA
pca = PCA(n_components=32)
pca.fit(X.cpu().numpy())
print(pca.explained_variance_ratio_)
print(np.cumsum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[6.04988444e-01 2.78767412e-01 3.75153636e-02 2.25193828e-02
 1.12964815e-02 6.94657920e-03 5.08627865e-03 3.77529854e-03
 3.33865184e-03 2.18388423e-03 2.06981816e-03 1.80349524e-03
 1.30467676e-03 1.02710134e-03 7.79518108e-04 7.35097953e-04
 7.08278784e-04 5.85514473e-04 5.30644356e-04 5.10878017e-04
 4.44775372e-04 3.95509345e-04 3.64871610e-04 3.49403639e-04
 3.00046191e-04 2.80978830e-04 2.73770155e-04 2.60689833e-04
 2.32596061e-04 2.23599177e-04 2.04460042e-04 1.87807033e-04]
[0.60498844 0.88375586 0.92127122 0.9437906  0.95508708 0.96203366
 0.96711994 0.97089524 0.97423389 0.97641778 0.97848759 0.98029109
 0.98159577 0.98262287 0.98340239 0.98413748 0.98484576 0.98543128
 0.98596192 0.9864728  0.98691758 0.98731308 0.98767796 0.98802736
 0.98832741 0.98860838 0.98888215 0.98914284 0.98937544 0.98959904
 0.9898035  0.98999131]
[3076.399    2088.2866    766.0792    593.5364    420.3787    329.65115
  282.07797   243.02153   228.53603   184.83487   179.9431    167.96802
  142.86